# Запусти, чтобы посмотреть конкретный заход

In [155]:
#### import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%matplotlib qt5
df = pd.read_csv('SCRLOG_0002.csv', sep =';')
df.columns = df.columns.str.strip()

#count = df['wind_correct.heading'].isnull().sum()
#df=df.drop([0, count-1]).reset_index(drop=True)
#df = df[:-count].reset_index(drop=True)

df = df.drop_duplicates().reset_index(drop=True)
df=df.reset_index(drop=True)
df
empty_numbers= df[df['wind_correct.heading'].isnull()].index.tolist()
#empty_rows = df[df.isnull().sum(axis=1)>0].index.tolist()
empty_rows = df.loc[empty_numbers]
#empty_rows_df = pd.DataFrame(empty_rows)
#new_c = ['purpose','p','i','d']
#empty_rows_df.columns = new_c

df = df.apply(pd.to_numeric, errors='coerce')#учла наличие пустых значений
empty_numbers.append(len(df))
print(empty_numbers)
#print(empty_rows)
df = df[['time_stamp(millis)','math.deg(current_orient:get_euler_pitch())','calc_target.pitch','wind_correct.pitch','absolute_aim_orientation.pitch','wrap_360(math.deg(current_orient:get_euler_yaw()))','calc_target.heading','wind_correct.heading','absolute_aim_orientation.bearing','ahrs:airspeed_estimate()', 'ctrl_roll_speed', 'ctrl_yawspd', 'ctrl_roll_speed']]

#####
#empty_rows = df[df.isnull().any(axis=1)].index.tolist()
#plt.figtext(0.02,0.02, empty_numbers, fontsize=10)
#plt.show()
#####


new = pd.DataFrame(columns=['wrap_360(math.deg(current_orient:get_euler_yaw()))','minus_absolute_aim_orientation.bearing'])
new['minus_wrap_360(math.deg(current_orient:get_euler_yaw()))'] = df['wrap_360(math.deg(current_orient:get_euler_yaw()))'] - df['calc_target.heading']
df['minus_wrap_360(math.deg(current_orient:get_euler_yaw()))'] = new['minus_wrap_360(math.deg(current_orient:get_euler_yaw()))']
new['minus_absolute_aim_orientation.bearing'] = df['absolute_aim_orientation.bearing'] - df['calc_target.heading']
df['minus_absolute_aim_orientation.bearing'] = new['minus_absolute_aim_orientation.bearing']
df['x']= df['calc_target.heading']*0
df['cam_width_left_6']= df['calc_target.heading']*0+6
df['cam_width_right_6']= df['calc_target.heading']*0-6
df['cam_width_left_14']= df['calc_target.heading']*0+14
df['cam_width_right_14']= df['calc_target.heading']*0-14

########### new difference
x_plane = pd.DataFrame(columns=['plane_part'])
y_plane = pd.DataFrame(columns=['plane_part'])
x_point = pd.DataFrame(columns=['point_part'])
y_point = pd.DataFrame(columns=['point_part'])
sinus_part = pd.DataFrame(columns=['sinus_part'])
cosinus_part = pd.DataFrame(columns=['cosinus_part'])
angle = pd.DataFrame(columns=['angle'])
angle_1 = pd.DataFrame(columns=['angle_1'])

df['r_minus_wrap_360(math.deg(current_orient:get_euler_yaw()))'] = np.radians(df['minus_wrap_360(math.deg(current_orient:get_euler_yaw()))'])

x_plane['plane_part'] = np.sin(df['r_minus_wrap_360(math.deg(current_orient:get_euler_yaw()))'])
y_plane['plane_part'] = np.cos(df['r_minus_wrap_360(math.deg(current_orient:get_euler_yaw()))'])

df['x'] = np.radians(df['x'])
x_point['point_part'] = np.sin(df['x'])
y_point['point_part'] = np.cos(df['x'])


sinus_part['sinus_part'] = x_plane['plane_part']*x_point['point_part'] + y_plane['plane_part']*y_point['point_part']
cosinus_part['cosinus_part'] = x_plane['plane_part']*y_point['point_part'] - x_point['point_part']*y_plane['plane_part']

angle['angle'] = np.arctan2(cosinus_part ['cosinus_part'],sinus_part['sinus_part'])
#angle['difference'] = angle['angle']**2
angle_1['angle_1'] = (angle['angle']**2).expanding().mean()
angle_1['angle_1'] = np.sqrt(angle_1['angle_1'])
df['difference'] = np.degrees(angle_1['angle_1'])

'''########### it was a new difference
rename = { 'math.deg(current_orient:get_euler_pitch())': 'purpose',
           'wrap_360(math.deg(current_orient:get_euler_yaw()))': 'p',
          'calc_target.pitch': 'i',
          'calc_target.heading': 'd'   
    }
print(empty_rows.loc['math.deg(current_orient:get_euler_pitch())','wrap_360(math.deg(current_orient:get_euler_yaw()))','calc_target.pitch','calc_target.heading'].rename(columns = rename))
'''
print("Какой из заходов на цель хотите посмотреть?")
num = int(input())
print ("Введите критерий: 0 - всe,1 - тангаж, 2 - курс,3 - билиберда?, 4 - отклонение")
kriteriy = input()
df1 = df.iloc[empty_numbers[num-1]+1:empty_numbers[num]]
##########
#plt.figtext(0.02, 0.02, empty_rows.iloc[:1])
#fig.subplotsadjust(hspace=0.3)
if (kriteriy == "всe" or kriteriy =="тангаж") or (kriteriy == "0" or kriteriy =="1"):
    df1.plot(x="time_stamp(millis)",y=["calc_target.pitch","wind_correct.pitch","absolute_aim_orientation.pitch"])
    plt.plot(df1["time_stamp(millis)"],df1["math.deg(current_orient:get_euler_pitch())"], color = "green", linestyle ='-',linewidth = 2, alpha = 0.8, label = 'math.deg(current_orient:get_euler_pitch())')
    #plt.plot(x2,y, color = "blue", linestyle ='-',linewidth = 1, alpha = 1, label = 'x2')
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left',fontsize=22)
    
#!!!!!!!!!!!!!!!!!!!
if ((kriteriy == "всe" or kriteriy == "курс")) or (kriteriy == "0" or kriteriy =="2"):
    #df1.plot(x="time_stamp(millis)",y=['minus_absolute_aim_orientation.bearing','wrap_360(math.deg(current_orient:get_euler_yaw()))','x','cam_width_right_6','cam_width_left_6','cam_width_right_14','cam_width_left_14'])
    df1.plot(x="time_stamp(millis)",y=['minus_absolute_aim_orientation.bearing','minus_wrap_360(math.deg(current_orient:get_euler_yaw()))'],linewidth = 3)
    plt.plot(df1["time_stamp(millis)"],df1['cam_width_right_14'], color = "red", linestyle ='--',linewidth = 4, alpha = 1, label = 'cam_width_right_14')
    plt.plot(df1["time_stamp(millis)"],df1['cam_width_left_14'], color = "red", linestyle ='--',linewidth = 4, alpha = 1, label = 'cam_width_left_14')
    plt.plot(df1["time_stamp(millis)"],df1['cam_width_right_6'], color = "black", linestyle ='--',linewidth = 3, alpha = 1, label = 'cam_width_right_6')
    plt.plot(df1["time_stamp(millis)"],df1['cam_width_left_6'], color = "black", linestyle ='--',linewidth = 3, alpha = 1, label = 'cam_width_left_6')
    plt.plot(df1["time_stamp(millis)"],df1['x'], color = "black", linestyle ='-',linewidth = 3, alpha = 1, label = 'x')
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left',fontsize=22)
    
if (kriteriy == "всe" or kriteriy == "билиберда")or (kriteriy == "0" or kriteriy =="3"):
    df1.plot(x="time_stamp(millis)",y=['ahrs:airspeed_estimate()', 'ctrl_roll_speed', 'ctrl_yawspd', 'ctrl_Roll'])
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left',fontsize=22)

if ((kriteriy == "всe" or kriteriy == "отклонение")) or (kriteriy == "0" or kriteriy =="4"):
    df1.plot(x='time_stamp(millis)',y='difference', color = "blue", linestyle ='-',linewidth = 3, alpha = 1, label = 'difference')
    plt.plot(df1["time_stamp(millis)"],df1['x'], color = "black", linestyle ='-',linewidth = 3, alpha = 1, label = 'x')
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left',fontsize=22)
'''print(empty_rows[1:5].rename(columns={
    'math.deg(current_orient:get_euler_pitch())': 'purpose',
    'wrap_360(math.deg(current_orient:get_euler_yaw()))': 'p',
    'calc_target.pitch': 'i',
    'calc_target.heading':'d'
 }))
print(empty_rows.iloc[4:5, 1:5])
#spisok = []
#spisok = empty_rows.iloc[num-1:num,1:5]
#print(spisok)
#########################################################################
print(empty_rows.iloc[1:5].rename(columns={
    'math.deg(current_orient:get_euler_pitch())': 'purpose',
    'wrap_360(math.deg(current_orient:get_euler_yaw()))': 'p',
    'calc_target.pitch': 'i',
    'calc_target.heading': 'd'
})[['purpose', 'p', 'i', 'd']]) 
plt.figtext(0.02, 0.9, empty_rows.iloc[num-1:num,1:5].to_string(),fontsize=22)'''
selected_data = empty_rows.iloc[num-1:num, 1:5].rename(columns={
    'math.deg(current_orient:get_euler_pitch())': 'purpose',
    'wrap_360(math.deg(current_orient:get_euler_yaw()))': 'p',
    'calc_target.pitch': 'i', 
    'calc_target.heading': 'd'
})
data_string = selected_data.to_string()

# Выводим в figtext
plt.figtext(0.02, 0.9, data_string, fontsize=22)
plt.figtext(0.03, 0.02, df1['difference'].iloc[-1], fontsize=22)
plt.show()
#df.loc[empty_numbers[num],'difference']
#5, p i d - 3,4,5 

[1, 4604, 8277, 8733, 13251, 17292, 22452, 22876, 26378, 27293, 28013, 33438, 36890, 38891, 42372, 45769]
Какой из заходов на цель хотите посмотреть?


 1


Введите критерий: 0 - всe,1 - тангаж, 2 - курс,3 - билиберда?, 4 - отклонение


 4


In [148]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45769 entries, 0 to 45768
Data columns (total 22 columns):
 #   Column                                                      Non-Null Count  Dtype  
---  ------                                                      --------------  -----  
 0   time_stamp(millis)                                          45768 non-null  float64
 1   math.deg(current_orient:get_euler_pitch())                  45768 non-null  float64
 2   calc_target.pitch                                           45768 non-null  float64
 3   wind_correct.pitch                                          45753 non-null  float64
 4   absolute_aim_orientation.pitch                              45753 non-null  float64
 5   wrap_360(math.deg(current_orient:get_euler_yaw()))          45768 non-null  float64
 6   calc_target.heading                                         45768 non-null  float64
 7   wind_correct.heading                                        45753 non-null  float64
 

# Запусти чтобы посмотреть все заходы в одном окне по времени

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib qt5
df = pd.read_csv('SCRLOG_02.csv', sep =';')
df.columns = df.columns.str.strip()

#count = df['wind_correct.heading'].isnull().sum()
#df=df.drop([0, count-1]).reset_index(drop=True)
#df = df[:-count].reset_index(drop=True)

df = df.drop_duplicates().reset_index(drop=True)
df=df[1:].reset_index(drop=True)

empty_numbers=df[df['wind_correct.heading'].isnull()].index.tolist()
df=df.dropna(axis=0, how='any')#не запустила в первом случае
df = df.apply(pd.to_numeric, errors='coerce')#учла наличие пустых значений
empty_numbers.append(len(df))

df = df[['time_stamp(millis)','math.deg(current_orient:get_euler_pitch())','calc_target.pitch','wind_correct.pitch','absolute_aim_orientation.pitch','wrap_360(math.deg(current_orient:get_euler_yaw()))','calc_target.heading','wind_correct.heading','absolute_aim_orientation.bearing','ahrs:airspeed_estimate()', 'ctrl_roll_speed', 'ctrl_yawspd', 'ctrl_Roll']]


df.plot(x="time_stamp(millis)",y=["math.deg(current_orient:get_euler_pitch())","calc_target.pitch","wind_correct.pitch","absolute_aim_orientation.pitch"])
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
df.plot(x="time_stamp(millis)",y=['wrap_360(math.deg(current_orient:get_euler_yaw()))','calc_target.heading','wind_correct.heading','absolute_aim_orientation.bearing'])
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
df1.plot(x="time_stamp(millis)",y=['ahrs:airspeed_estimate()', 'ctrl_roll_speed', 'ctrl_yawspd', 'ctrl_Roll'])
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.show()

'ctrl_yawspd' в радианы
умножить ahrs:airspeed_estimate()
/9.806565
от результата арктангенс
результат в градусы
график - таргкт ролл

наложить ctrl_Roll

In [13]:
new_roll = np.arctan(np.radians(df['ctrl_yawspd'])*df['ahrs:airspeed_estimate()'])
new_roll

NameError: name 'np' is not defined

In [3]:
print(df.columns.tolist())

['time_stamp(millis)', 'math.deg(current_orient:get_euler_pitch())', 'calc_target.pitch', 'wind_correct.pitch', 'absolute_aim_orientation.pitch', 'wrap_360(math.deg(current_orient:get_euler_yaw()))', 'calc_target.heading', 'wind_correct.heading', 'absolute_aim_orientation.bearing']
